# Analyzing energy consumption of the world.


#### author: bhavesh patel

In [2]:
# load world energy data.
# Source: http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls

def load_energy_data():

    import pandas as pd
    import re

    # I could use lambda as follows but too many things to check.  So creating separate function: data_cleanup.
    # energy = pd.read_excel('Energy Indicators.xls', header=1, skiprows=16, skip_footer=38, na_values='...',
    #                       converters = {'Unnamed: 2' : lambda x : 
    #                                     "South Korea" if x=="Republic of Korea" else
    #                                     "United States" if x=="United States of America20" else
    #                                     "United Kingdom" if x== "United Kingdom of Great Britain and Northern Ireland" else
    #                                     "Hong Kong" if x=="China, Hong Kong Special Administrative Region" else
    #                                     x
    #                                    })

    def data_cleanup(x):
        # first strip additional numbers at the end.
        x = x.rstrip('1234567890')  # remove footnote numbers associated with some countries.
        x = re.sub('\(.+\)','',x)   # remove other info like 'Bolivia (Plurinational State of)' should be 'Bolivia',
        x = x.rstrip()              # remove blanks at the end of a string
        if x=="Republic of Korea":
            return "South Korea"
        elif x=="United States of America":
            return "United States"
        elif x== "United Kingdom of Great Britain and Northern Ireland": 
            return "United Kingdom"
        elif x=="China, Hong Kong Special Administrative Region":
            return "Hong Kong"
        else:
            return x


    # read excel and skip first 15 rows, line 16 is a header, skip 38 rows in footer.

    # I tried converting energy supply Gigjoules in the same forumla using: 
    # converters={'Petajoules' : lambda x : x*1000000}.  This ran into an issue."
    # The "..." should convert to NaN.  But that didn't work, as it converted "..." to "..." times 1000000!!!

    energy = pd.read_excel('http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls', 
                           sheetname='Energy',header=1, skiprows=16, skip_footer=38, na_values='...',
                            converters = {'Unnamed: 2' : lambda x : data_cleanup(x)})

    #rename columns,
    energy.columns= ['one','two','Country','Energy Supply', 'Energy Supply per Capita', '% Renewable']

    #get rid of first two columns.
    energy = energy[['Country','Energy Supply', 'Energy Supply per Capita', '% Renewable']]

    # Convert to Gigajoules.
    energy['Energy Supply'] = energy['Energy Supply'] * 1000000

    return energy

energy=load_energy_data()

energy

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:661)>

In [ ]:
def load_gdp_data():
    # load world's gdp data.
    # Source: http://data.worldbank.org/indicator/NY.GDP.MKTP.CD

    import pandas as pd

    gdp = pd.read_excel('http://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=excel', 
                        sheetname='Data',header=1, skiprows=2,
                        converters = {'Country Name' : lambda x : 
                                      "South Korea" if x=="Korea, Rep." else
                                      "Iran" if x=="Iran, Islamic Rep." else
                                      "Hong Kong" if x=="Hong Kong SAR, China" else
                                      x
                                     })

    # Use only the last 10 years (2006-2015) of GDP data.
    gdp_current_years=gdp[list(gdp.columns[0:1]) + list(gdp.columns[50:])]

    # drop year 2016 as it has no values.
    # We can use drop command.  Remember, axis=0 for rows and axis=1 for columns.
    gdp_current_years=gdp_current_years.drop('2016', axis=1)
    gdp_current_years=gdp_current_years.rename(columns={'Country Name':'Country'})
    #gdp_current_years
    return gdp_current_years

gdp_current_years=load_gdp_data()

gdp_current_years

In [ ]:
def load_scimago_data():
    import pandas as pd
    # now load Scimago Journal & Country Rank data.
    # http://www.scimagojr.com/countryrank.php?category=2102

    scima_country_rank = pd.read_excel('http://www.scimagojr.com/countryrank.php?category=2102&out=xls',
                                       sheetname='Sheet1')

    return scima_country_rank

scima_country_rank=load_scimago_data()

scima_country_rank

In [ ]:
def join_data():
    import pandas as pd
    
    # Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). 
    # Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 
    #'Rank' (Rank 1 through 15).

    rank_energy = pd.merge(scima_country_rank,energy, how='inner', left_on='Country', right_on='Country')
    #The index of this DataFrame should be the name of the country.
    rank_energy_gdp = pd.merge(rank_energy, gdp_current_years, how='inner', 
                               left_on='Country', right_on='Country').set_index(['Country'])

    return rank_energy_gdp[0:15]

rank_energy_gdp=join_data()

rank_energy_gdp


The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [ ]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [ ]:
def union_data():
    import pandas as pd
    # let's find out the union of all 3.
    # Subtract the intersect of all 3 from union. That will give us how many entries we removed.

    # Union
    rank_energy_unioun = pd.merge(scima_country_rank,energy, how='outer', left_on='Country', right_on='Country')
    rank_energy_gdp_unioun = pd.merge(rank_energy_unioun, gdp_current_years, how='outer', left_on='Country', right_on='Country').set_index(['Country'])

    # Calculate interset again, as its part of answer_one() function, not global.
    rank_energy_intetsect = pd.merge(scima_country_rank,energy, how='inner', left_on='Country', right_on='Country')
    rank_energy_gdp_intetsect = pd.merge(rank_energy_intetsect, gdp_current_years, how='inner', 
                               left_on='Country', right_on='Country').set_index(['Country'])

    # Nunber of entries we removed are:
    ent_removed=rank_energy_gdp_unioun.shape[0]-rank_energy_gdp_intetsect.shape[0]
    return ent_removed

union_data()


What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [ ]:
def avg_gdp():
    import pandas as pd
    import numpy as np
    def avgval(data):
        sel_columns=data[['2006',
                         '2007',
                         '2008',
                         '2009',
                         '2010',
                         '2011',
                         '2012',
                         '2013',
                         '2014',
                         '2015']]
        return pd.Series({'avgGDP0615': np.mean(sel_columns)})

    avgGDP=rank_energy_gdp.apply(avgval, axis=1)
    avgGDPSorted = avgGDP.sort_values(['avgGDP0615'], ascending=False)
    return avgGDPSorted['avgGDP0615']

avgGDP=avg_gdp()
#type(avgGDP)
avgGDP

By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

In [ ]:
def gdp_change():
    import pandas as pd
    gdpchgcountry = pd.DataFrame(avgGDP)
    gdpchgcountry['Country']=gdpchgcountry.index
    x=gdpchgcountry.iloc[5]['Country']
    no5gdp=rank_energy_gdp[rank_energy_gdp.index==x]

    chg=no5gdp['2015']-no5gdp['2006']    
    return chg[0]
gdpchg6=gdp_change()

gdpchg6

What is the mean `Energy Supply per Capita`?

In [ ]:
def mean_energy_per_capita():
    import numpy as np
    return np.mean(rank_energy_gdp['Energy Supply per Capita'])

meanengpercap = mean_energy_per_capita()

meanengpercap

What country has the maximum % Renewable and what is the percentage?

In [ ]:
def most_renewable():
    rank_energy_sorted=rank_energy_gdp.sort_values(['% Renewable'], ascending=False)
    country=rank_energy_sorted.index[0]
    topren=rank_energy_sorted['% Renewable'][0]
    return (country, topren)

topren= most_renewable()

topren

Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

In [ ]:
def max_citation():    
    rank_energy_gdp['SelfCitationRatio']=rank_energy_gdp['Self-citations']/rank_energy_gdp['Citations']
    rank_energy_gdp.sort_values(['SelfCitationRatio'], ascending=False)
    country=rank_energy_gdp.index[0]
    topren=rank_energy_gdp['SelfCitationRatio'][0]
    return (country, topren)

topren=max_citation()
topren

What is the third most populous country according to this estimate?


In [ ]:
def most_populous_country():
    rank_energy_gdp['Estpop']=rank_energy_gdp['Energy Supply']/rank_energy_gdp['Energy Supply per Capita']
    rank_energy_sorted=rank_energy_gdp.sort_values(['Estpop'], ascending=False)
    country=rank_energy_sorted.index[2]
    return country

topren=most_populous_country()
topren

Let's estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

In [ ]:
def pearson_co():
    rank_energy_gdp['EstCitperPerson']=rank_energy_gdp['Citable documents']/rank_energy_gdp['Estpop']
    return rank_energy_gdp['EstCitperPerson'].corr(rank_energy_gdp['Energy Supply per Capita'])

corrval=pearson_co()
corrval

In [ ]:
def plot_pearson():
    import matplotlib as plt
    %matplotlib inline
    rank_energy_gdp.plot(x='EstCitperPerson', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])
    
plot_pearson()

Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.


In [ ]:
def country_renewable_rank():
    import numpy as np
    import pandas as pd
    renMedian=np.median(rank_energy_gdp['% Renewable'])
    rank_energy_gdp['RenEnergyState']=rank_energy_gdp.apply(lambda row: 1 if row['% Renewable'] >= renMedian else 0, axis=1)
    rank_energy_sorted = rank_energy_gdp.sort_values(['Rank'])
    #HighRenew = pd.Series(rank_energy_sorted.index)
    return rank_energy_sorted['Rank']

country_list=country_renewable_rank()

#type(country_list)
country_list

Get energy usage by continent.


In [ ]:
def continent_energy_use():    
    import pandas as pd
    import numpy as np
    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}

    rank_energy_gdp['Country']=rank_energy_gdp.index
    rank_energy_gdp['Continent']=rank_energy_gdp.apply(lambda row: ContinentDict[row['Country']], axis=1)

    return (rank_energy_gdp.set_index('Continent')
                    .groupby(level=0)['Estpop']
                     .agg({'size' : np.count_nonzero,
                           'sum' : np.sum,
                           'mean' : np.average,
                           'std' : np.std
                          }))

ans_ele=continent_energy_use()

ans_ele

Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?


In [ ]:
def top_15():
    import pandas as pd
    import numpy as np

    rank_energy_gdp['renewablecategory']=pd.cut(rank_energy_gdp['% Renewable'],5)
    rank_sorted=rank_energy_gdp.sort_values(['Continent','renewablecategory'], ascending=[True,True])
    return rank_sorted.groupby(['Continent', 'renewablecategory'])['Country'].count()

top15Cont=top_15()
#type(ans_12)
top15Cont

Convert the Population Estimate series to a string with thousands separator (using commas). 


In [ ]:
def pop_estimate():
    return rank_energy_gdp['Estpop'].apply(lambda row: '{:,}'.format(row))

population_estimate=pop_estimate()
population_estimate

Visulization of the data!

In [ ]:
def plot_energy():
    import matplotlib as plt
    %matplotlib inline
    #Top15 = answer_one()
    ax = rank_energy_gdp.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*rank_energy_gdp['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(rank_energy_gdp.index):
        ax.annotate(txt, [rank_energy_gdp['Rank'][i], rank_energy_gdp['% Renewable'][i]], ha='center')

    print("Visualization: % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")
    
plot_energy()